In [ ]:
import os

if not os.path.isdir("cld_optimization_experiments"):
    !git clone https://github.com/oopir/cld_optimization_experiments

%cd cld_optimization_experiments

In [ ]:
import numpy as np
import random
import torch

from src.data import load_digits_data
from src.training import train_multiseed
from src.plots import plot_ex1_multiseed, plot_ex1_multiseed_short

torch.cuda.empty_cache()
device = "cuda" if torch.cuda.is_available() else "cpu"
data_seed = 0
run_seeds = list(range(1))

In [ ]:
n = 10
betas_to_plot = [10*n, 50*n, 100*n]
m = max([n * np.log(n) * (beta) * np.log(beta) for beta in betas_to_plot])
m = int(max(4096,m))
print(f"m={m}")

In [ ]:
epochs = 250000
eta    = 1e-4
jac_probe_size = 10
track_every = max(1,epochs//100) # track at most 100 times
print_every = epochs//10

In [ ]:
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

# data = load_digits_data(n=n, random_labels=False, device=device, seed=data_seed)
# d = data["X_train"].shape[1]

common = dict(
    seeds=run_seeds,
    n=n,
    eta=eta, 
    epochs=epochs,
    m=m,
    init_type="standard",
    regularization_scale=1.0,
    track_jacobian=True, 
    jac_probe_size=jac_probe_size,
    use_linearized=True, # for now plotting has bugs when this is False
    device=device,
    track_every=track_every,
    print_every=print_every,
)

results = {f"β={beta//n}n": train_multiseed(beta=beta, **common,) for beta in betas_to_plot}

In [ ]:
plot_ex1_multiseed_short(results, epochs, track_every)

In [ ]:
# plot_ex1_multiseed(results, epochs, track_every)